<a href="https://colab.research.google.com/github/faisalnawazmir/Econometrics-ML_for_Finance/blob/main/ayesha__Fama_6_Factor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fama French Six Factor Model

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm 
from tabulate import tabulate
import time
np.random.seed(12345)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [34]:
#copying data from excel file 25 portfoli
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
file = r'/gdrive/My Drive/data/Factors FF_ayesha.xlsx'
df = pd.read_excel(file)
df.head()

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


,Date,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p25,Mkt_rf,SMB,HML,WML,CMA,RMW,Reg
0,2000-01-01,0.178104,0.149602,0.142498,0.121144,0.110616,0.117735,0.110664,0.092240,0.068990,0.076588,0.064833,0.058543,0.049305,0.035943,0.026366,0.031131,0.017331,0.001694,-0.009736,-0.014255,-0.015920,-0.021284,-0.008742,-0.037421,-0.035024,-0.040109,-0.009489,0.007305,0.029458,0.015903,0.003310,-0.040229
1,2000-01-02,0.092635,0.107413,0.108822,0.108591,0.111967,0.111211,0.119137,0.116468,0.120271,0.116661,0.101254,0.098793,0.109843,0.072183,0.070388,0.074738,0.064502,0.077777,0.083013,0.083013,0.104342,0.075977,0.085660,0.080385,0.090332,-0.035767,-0.005061,0.017978,0.036510,0.018628,0.008605,-0.016751
2,2000-01-03,0.132933,0.111175,0.102088,0.092935,0.091619,0.087737,0.089550,0.080550,0.045984,0.025300,0.020605,0.034820,0.055351,0.061460,0.068001,0.070566,0.112957,0.103677,0.108457,0.120620,0.116043,0.125302,0.154267,0.093937,0.108526,-0.081055,0.013123,0.017057,0.031607,0.007759,0.001593,0.187280
3,2000-01-04,0.019185,0.023567,0.046560,0.033526,0.054471,0.060460,0.068659,0.065226,0.065841,0.024099,0.018128,0.013584,0.007942,0.003133,-0.027661,-0.019598,-0.010939,0.000233,-0.005809,-0.002867,0.013325,0.008685,0.025226,0.049480,0.052285,0.022559,0.019400,0.022753,0.043048,0.022607,0.016058,-0.003890
4,2000-01-05,0.004970,-0.007078,-0.009864,-0.000072,0.003017,0.014748,0.014504,0.016163,0.008998,0.035545,0.028835,0.036682,0.041242,0.036976,0.053052,0.062109,0.054398,0.057881,0.079518,0.072934,0.087000,0.090995,0.122150,0.112607,0.164821,-0.001434,0.011741,0.019596,0.040089,0.014866,0.023866,0.046271


In [35]:
#N = 25  # of firms in an economy, here # of portfolios, here len(data)
T = len(df)  # length of periods # here 1136#df.columns.array
#  data
data = df.loc[:,'p1':'p25']
#data.shape#(T,N)
#x,y=data.shape
#N=y#  # of firms in an economy, here # of portfolios, here len(data)
#T = data.shape[0]
N = data.shape[1]

In [36]:
# simulate factor loadings (ie, coefficients)
b_mkt = np.zeros((1,N)) # true loadings for market factor
b_smb = np.zeros((1,N)) # true loadings for size factor
b_hml = np.zeros((1,N)) # true loadings for value factor
b_cma = np.zeros((1,N)) # true loadings for value factor
b_rmw = np.zeros((1,N)) # true loadings for value factor
b_umd=np.zeros((1,N)) # true loadings for value factor

In [37]:
#  factor returns
mkt = df['Mkt_rf'] # market factor
smb = df['SMB'] # size factor
hml = df['HML'] # value factor
cma = df['CMA'] # market factor
rmw = df['RMW'] # size factor
umd=df['WML']# umd is growth factor


In [38]:
#data.iloc[:,1]
fctrs = df[["Mkt_rf","SMB","HML","CMA","RMW","WML"]]
fctrs.head()


,Mkt_rf,SMB,HML,CMA,RMW,WML
0,-0.040109,-0.009489,0.007305,0.015903,0.003310,0.029458
1,-0.035767,-0.005061,0.017978,0.018628,0.008605,0.036510
2,-0.081055,0.013123,0.017057,0.007759,0.001593,0.031607
3,0.022559,0.019400,0.022753,0.022607,0.016058,0.043048
4,-0.001434,0.011741,0.019596,0.014866,0.023866,0.040089


In [39]:
no_factors=fctrs.shape[1]

In [40]:
# Step 1: time series regression firm by firm
#fctrs = np.c_[mkt, smb, hml]
# of factors
b = np.zeros(no_factors)
for i in range(N): # firm-by-firm time series regression
  model = LinearRegression()
  model.fit(fctrs,data.iloc[:,i])
  b = np.c_[b,model.coef_]

b  = np.transpose(b[:,1:])
b.shape
#b should be 0.1, 0.2

(25, 6)

In [41]:
#b is bhat
#b
#b.shape# (N,#factors)
df_bhat = pd.DataFrame(b)
df_bhat.columns = ['b_01','b_02','b_03','b_04','b_05','b_06']# change according factos
#print(df_bhat.head())

from google.colab import  drive
df_bhat.to_csv('/gdrive/My Drive/data/bhat_ayesha_6factors.csv')

In [42]:
# Step 2: cross sectional regression time-by-time
f = np.zeros(no_factors)
for i in range(T): # time-by-time cross sectional regression
  model = LinearRegression()
  model.fit(b,data.iloc[i,:])
  f = np.c_[f,model.coef_]

f = np.transpose(f[:,1:])
df_gamma = pd.DataFrame(f)
df_gamma.columns = ['mkt','smb','hml','cma','rmw','umd']
print(df_gamma.describe())
print('Median estimate:', np.median(f,axis=0))
print('Standard errors (without Shanken correction):', np.std(f,axis=0)/T**.5)

              mkt         smb         hml         cma         rmw         umd
count  240.000000  240.000000  240.000000  240.000000  240.000000  240.000000
mean     0.013695   -0.002387   -0.008973   -0.002266    0.001594    0.013876
std      0.391197    0.102028    0.197719    0.163688    0.153887    0.203843
min     -1.612401   -0.505936   -0.683310   -0.457751   -0.470624   -0.732854
25%     -0.279039   -0.066171   -0.134508   -0.098293   -0.092688   -0.114621
50%      0.039587    0.001391    0.005737   -0.019197   -0.011903    0.013195
75%      0.290835    0.064975    0.110178    0.090622    0.092030    0.140601
max      0.953261    0.393729    0.490249    0.652990    0.546596    0.713661
Median estimate: [ 0.03958745  0.00139073  0.00573665 -0.019197   -0.01190271  0.01319498]
Standard errors (without Shanken correction): [0.02519902 0.00657213 0.0127361  0.01054397 0.00991263 0.01313057]


In [43]:
mu=np.median(f,axis=0)
sigma=(np.std(f,axis=0))/T**.5
tstat=mu/sigma
tstat
#tstat<1.96

array([ 1.57099174,  0.21161007,  0.45042443, -1.82066089, -1.20076235,
        1.00490494])

In [44]:
f.shape # (T,#factors)# f or df I assume is gamma hat
df_gamma.to_csv('/gdrive/My Drive/data/ayesha_gamma_6factors.csv')

In [45]:
df.head()#f and df are same, gamma hat

,Date,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p25,Mkt_rf,SMB,HML,WML,CMA,RMW,Reg
0,2000-01-01,0.178104,0.149602,0.142498,0.121144,0.110616,0.117735,0.110664,0.092240,0.068990,0.076588,0.064833,0.058543,0.049305,0.035943,0.026366,0.031131,0.017331,0.001694,-0.009736,-0.014255,-0.015920,-0.021284,-0.008742,-0.037421,-0.035024,-0.040109,-0.009489,0.007305,0.029458,0.015903,0.003310,-0.040229
1,2000-01-02,0.092635,0.107413,0.108822,0.108591,0.111967,0.111211,0.119137,0.116468,0.120271,0.116661,0.101254,0.098793,0.109843,0.072183,0.070388,0.074738,0.064502,0.077777,0.083013,0.083013,0.104342,0.075977,0.085660,0.080385,0.090332,-0.035767,-0.005061,0.017978,0.036510,0.018628,0.008605,-0.016751
2,2000-01-03,0.132933,0.111175,0.102088,0.092935,0.091619,0.087737,0.089550,0.080550,0.045984,0.025300,0.020605,0.034820,0.055351,0.061460,0.068001,0.070566,0.112957,0.103677,0.108457,0.120620,0.116043,0.125302,0.154267,0.093937,0.108526,-0.081055,0.013123,0.017057,0.031607,0.007759,0.001593,0.187280
3,2000-01-04,0.019185,0.023567,0.046560,0.033526,0.054471,0.060460,0.068659,0.065226,0.065841,0.024099,0.018128,0.013584,0.007942,0.003133,-0.027661,-0.019598,-0.010939,0.000233,-0.005809,-0.002867,0.013325,0.008685,0.025226,0.049480,0.052285,0.022559,0.019400,0.022753,0.043048,0.022607,0.016058,-0.003890
4,2000-01-05,0.004970,-0.007078,-0.009864,-0.000072,0.003017,0.014748,0.014504,0.016163,0.008998,0.035545,0.028835,0.036682,0.041242,0.036976,0.053052,0.062109,0.054398,0.057881,0.079518,0.072934,0.087000,0.090995,0.122150,0.112607,0.164821,-0.001434,0.011741,0.019596,0.040089,0.014866,0.023866,0.046271


In [46]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import statsmodels.stats.api as sms

In [47]:
data_a=data.to_numpy()
fctrs_a=fctrs.to_numpy()

In [51]:
# Compute standard error with bootstrapping (Shanken correction unnecessary)
# One simple bootstrapping 

indices = np.random.randint(low=0,high=T, size=T)
new_data = data_a[indices,:]
new_fctrs = fctrs_a[indices,:]
xx = indices[0]
print(xx)
print(new_data[0,:])
print(data_a[xx,:])


146
[0.19659028 0.14938676 0.12312175 0.10458853 0.09241206 0.06785229
 0.0759601  0.0795157  0.07748829 0.06570412 0.05692805 0.04332813
 0.03824793 0.04702481 0.03592178 0.03487356 0.02516866 0.02749468
 0.02521071 0.0237959  0.01980243 0.03693056 0.04543212 0.05227669
 0.04125802]
[0.19659028 0.14938676 0.12312175 0.10458853 0.09241206 0.06785229
 0.0759601  0.0795157  0.07748829 0.06570412 0.05692805 0.04332813
 0.03824793 0.04702481 0.03592178 0.03487356 0.02516866 0.02749468
 0.02521071 0.0237959  0.01980243 0.03693056 0.04543212 0.05227669
 0.04125802]


In [49]:
def efhats(data,fctrs): # function to compute average estimated factor return
  T = data.shape[0]
  N = data.shape[1]
  # Step 1: Time series regression
  x = np.c_[np.ones(T),fctrs]
  M = np.linalg.inv(x.T @ x) @ x.T
  bhat = (M @ data)[1:,:].T

  # Step 2: cross sectional regression
  # Include other control variables here (eg, firm characteristics)
  x = np.c_[np.ones(N),bhat]
  M = np.linalg.inv(x.T @ x) @ x.T
  fhat = (M @ data.T)[1:,:].T
  df = pd.DataFrame(fhat)
  df.columns = ['mkt','smb','hml','cma','rmw','WML']
  E_fhat = np.average(fhat,axis=0)
  return E_fhat 


In [52]:
E_fhat = efhats(data_a,fctrs_a)
simn = 10**3 # of bootstrapping
t = time.time() # bootstrapping start time 
for i in range(simn): 
  indices = np.random.randint(low=0,high=T,size=T) # resample by rows
  new_data = data_a[indices,:] # resampled stock return data
  new_fctrs = fctrs_a[indices,:] # resampled factor return data
  E_fhat = np.c_[E_fhat,efhats(new_data,new_fctrs)]
  # to check bootstrapping status
  if i % 100 == 0: print(i)

print('Simulation duration:', time.time()-t, 'seconds.')
df = pd.DataFrame(E_fhat.T)
df.columns = ['mkt','smb','hml','cma','rmw','WML']
print(df.describe())

0
100
200
300
400
500
600
700
800
900
Simulation duration: 1.029529333114624 seconds.
               mkt          smb  ...          rmw          WML
count  1001.000000  1001.000000  ...  1001.000000  1001.000000
mean      0.003498     0.000444  ...     0.001711     0.007235
std       0.024845     0.005292  ...     0.008297     0.012412
min      -0.120531    -0.014478  ...    -0.030291    -0.034998
25%      -0.011665    -0.002989  ...    -0.003367    -0.000444
50%       0.002718     0.000445  ...     0.001677     0.007154
75%       0.020409     0.003802  ...     0.006808     0.015275
max       0.095880     0.021470  ...     0.033215     0.048248

[8 rows x 6 columns]


In [53]:
print('FM regression esimate of expeced factor returns: \n', np.average(E_fhat.T,axis=0))
print('FM regression esimate of standard errors: \n', np.std(E_fhat.T,axis=0))

FM regression esimate of expeced factor returns: 
 [ 0.00349785  0.00044426 -0.00259245 -0.00010904  0.00171064  0.00723501]
FM regression esimate of standard errors: 
 [0.02483236 0.00528934 0.00979776 0.00975433 0.00829279 0.0124056 ]


In [24]:
#bootstrap t stats
mu_boot=np.average(E_fhat.T,axis=0)
sigma_boot=np.std(E_fhat.T,axis=0)
tstat_boot=mu_boot/sigma_boot
tstat_boot

array([ 0.09297974,  0.2516258 , -0.20584187, -0.00120543,  0.32472047,
       -0.19321223])

https://quant.stackexchange.com/questions/35781/how-to-interpret-the-grs-f-test-values
#grs test
https://stackoverflow.com/questions/24074481/fama-macbeth-regression-in-python-pandas-or-statsmodels

In [ ]:
pip install rpy2

In [ ]:
%load_ext rpy2.ipython

In [55]:
%%R -i df -w 5 -h 5 --units in -r 200
# import df from global environment
# make default figure size 5 by 5 inches with 200 dpi resolution

install.packages("GRS.test", repos='http://cran.us.r-project.org', quiet=TRUE)
library(GRS.test)


In [56]:
%%R -i data,fctrs

GRS.MLtest(data, fctrs)

$GRS.stat
          GRS
[1,] 1.110444

$GRS.pval
           GRS
[1,] 0.3327915

$thetas
         [,1]
[1,] 1.285864

$theta
         [,1]
[1,] 1.162049

$ratio
          [,1]
[1,] 0.9037107



In [57]:
%%R -o model -o coef
model<-GRS.MLtest(data, fctrs)
coef<-model$GRS.stat

In [58]:
print(model)

$GRS.stat
          GRS
[1,] 1.110444

$GRS.pval
           GRS
[1,] 0.3327915

$thetas
         [,1]
[1,] 1.285864

$theta
         [,1]
[1,] 1.162049

$ratio
          [,1]
[1,] 0.9037107




W statistic given in (7) of GRS (1989) <DOI:10.2307/1913625>

GRS.stat=GRS test statistic

GRS.pval=its p-value

theta=maximum Sharpe ratio of the K factor portfolios

thetas=slope of the efficient frontier based on all assets

ratio=theta/thetas, proportion of the potential efficiency

References
Gibbons, Ross, Shanken, 1989. A test of the efficiency of a given portfolio, Econometrica, 57,1121-
1152. <DOI:10.2307/1913625>


https://towardsdatascience.com/guide-to-r-and-python-in-a-single-jupyter-notebook-ff12532eb3ba

https://www.linkedin.com/pulse/interfacing-r-from-python-3-jupyter-notebook-jared-stufft/
how to switch R and python


In [59]:
print(coef)

[[1.11044351]]
